In [10]:
import os
import pandas as pd
import numpy as np
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
input_data_folder = "../input_files"
output_data_folder = "../input_files"
in_filename_csv = "sepsis_cases_1.csv"

Therefore, when training a tree, it is possible to compute how much each feature decreases the impurity. The more a feature decreases the impurity, the more important the feature is. In random forests, the impurity decrease from each feature can be averaged across trees to determine the final importance of the variable.

Reference  for the Code:

https://chrisalbon.com/machine_learning/trees_and_forests/feature_selection_using_random_forest/

https://towardsdatascience.com/random-forest-in-python-24d0893d51c0

In [12]:
data = pd.read_csv(os.path.join(input_data_folder, in_filename_csv), sep=",")

#data = data.dropna()

In [13]:
#Removing the Columns which is not necessary

columns_to_drop = ['Case ID', 'Complete Timestamp', 'label', 'user', 'event_nr', 'month', 'weekday', 'hour']

data = data.drop(columns_to_drop, axis=1)

#Get the Numeric Columns so that Columns which is Catorical can be filtered out

data_list = data.columns

data_num_cols = data._get_numeric_data().columns

data_cat_cols = list(set(data_list) - set(data_num_cols))

In [14]:
#data = data[~data.Diagnose.isin(['missing']) & ~data.DiagnosticArtAstrup.isin(['missing'])]
#df = df[df.line_race != 0]

In [15]:
#for label encoding
data_cat_dict = {}
for i in range(len(data_cat_cols)):

    _label = data_cat_cols[i]
    temp_list = data[[_label]].values.tolist()
    subsec_set = {(x[0]) for x in temp_list}
    
    print("subsec_set :", subsec_set)
    subsec_set = sorted(list(subsec_set))
    _index = dict()

    for ix, _ in enumerate(subsec_set):
        print("subsec_set enumerate :", subsec_set[ix])
        _index[subsec_set[ix]] = ix + 1
    
    _idx = lambda x: _index[x[_label]]
    
    #Actual Vlaues in the dictionary
    data_cat_dict[_label] = _index

    data[_label] = data.apply(_idx, axis=1)

subsec_set : {'False', 'missing', 'True'}
subsec_set enumerate : False
subsec_set enumerate : True
subsec_set enumerate : missing
subsec_set : {'False', 'missing', 'True'}
subsec_set enumerate : False
subsec_set enumerate : True
subsec_set enumerate : missing
subsec_set : {'False', 'missing', 'True'}
subsec_set enumerate : False
subsec_set enumerate : True
subsec_set enumerate : missing
subsec_set : {'QE', 'C', 'DB', 'CB', 'B', 'AE', 'HE', 'IC', 'FB', 'EC', 'RC', 'YD', 'RA', 'M', 'GA', 'EB', 'DC', 'I', 'other', 'TB', 'CE', 'VC', 'O', 'R', 'YA', 'LD', 'HD', 'U', 'FD', 'QD', 'AA', 'MC', 'VB', 'EA', 'MA', 'T', 'V', 'JD', 'JE', 'KE', 'D', 'Q', 'GC', 'VA', 'HC', 'OE', 'OB', 'ZA', 'Y', 'A', 'YB', 'JC', 'LC', 'XB', 'AC', 'ZD', 'AB', 'SA', 'SC', 'BD', 'PC', 'ID', 'HB', 'UC', 'FE', 'YC', 'LB', 'H', 'BA', 'BB', 'KB', 'LE', 'ZB', 'ME', 'W', 'CA', 'OC', 'JA', 'HA', 'ED', 'AD', 'MD', 'SB', 'E', 'ND', 'XD', 'ZC', 'TD', 'N', 'PB', 'PD', 'KA', 'BC', 'RB', 'GD', 'GE', 'P', 'TC', 'NC', 'F', 'KC', 'VD', 

In [7]:
data_cat_dict['Activity']

{'Admission IC': 1,
 'Admission NC': 2,
 'CRP': 3,
 'ER Registration': 4,
 'ER Sepsis Triage': 5,
 'ER Triage': 6,
 'IV Antibiotics': 7,
 'IV Liquid': 8,
 'LacticAcid': 9,
 'Leucocytes': 10,
 'Release A': 11,
 'Release B': 12,
 'Release C': 13,
 'Release D': 14,
 'other': 15}

In [8]:
data.describe()

,Diagnose,DiagnosticArtAstrup,DiagnosticBlood,DiagnosticECG,DiagnosticIC,DiagnosticLacticAcid,DiagnosticLiquor,DiagnosticOther,DiagnosticSputum,DiagnosticUrinaryCulture,...,SIRSCriteria2OrMore,Age,Activity,CRP,LacticAcid,Leucocytes,timesincemidnight,timesincelastevent,timesincecasestart,open_cases
count,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,...,13120.000000,13120.000000,13120.000000,13120.000000,13120.00000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000
mean,43.863720,1.353887,1.881250,1.816921,1.911738,1.863796,1.015473,1.016006,1.037805,1.522637,...,1.922713,71.852515,6.444131,85.531631,1.23532,10.151349,664.215396,662.994221,4780.640769,66.954878
std,38.051221,0.488756,0.338926,0.399733,0.301152,0.357597,0.159526,0.161138,0.215853,0.509628,...,0.285540,15.083803,3.225157,94.263823,1.28786,13.709408,341.461903,1584.321171,11097.414178,22.162125
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,20.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,12.000000,1.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,65.000000,3.000000,0.000000,0.00000,1.400000,360.000000,0.000000,22.266667,53.000000
50%,33.000000,1.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,...,2.000000,75.000000,6.000000,58.000000,1.20000,9.300000,590.000000,3.483333,302.633333,77.000000
75%,67.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,...,2.000000,85.000000,10.000000,137.000000,1.80000,13.900000,930.000000,410.000000,5088.970833,84.000000
max,134.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,90.000000,15.000000,573.000000,14.90000,381.300000,1439.000000,27525.000000,164682.816667,93.000000


In [9]:
data[:5]

,Diagnose,DiagnosticArtAstrup,DiagnosticBlood,DiagnosticECG,DiagnosticIC,DiagnosticLacticAcid,DiagnosticLiquor,DiagnosticOther,DiagnosticSputum,DiagnosticUrinaryCulture,...,SIRSCriteria2OrMore,Age,Activity,CRP,LacticAcid,Leucocytes,timesincemidnight,timesincelastevent,timesincecasestart,open_cases
0,1,2,2,2,2,2,1,1,1,2,...,2,85.0,4,0.0,0.0,0.0,555,0.000000,0.000000,81
1,1,2,2,2,2,2,1,1,1,2,...,2,85.0,10,0.0,0.0,9.6,567,0.000000,11.316667,81
2,1,2,2,2,2,2,1,1,1,2,...,2,85.0,3,21.0,0.0,9.6,567,0.000000,11.316667,81
3,1,2,2,2,2,2,1,1,1,2,...,2,85.0,9,21.0,2.2,9.6,567,11.316667,11.316667,81
4,1,2,2,2,2,2,1,1,1,2,...,2,85.0,6,21.0,2.2,9.6,573,6.616667,17.933333,81


In [56]:
# Labels are the values we want to predict
labels_activity = np.array(data['Activity'])

# Remove the labels from the features
# axis 1 refers to the columns
data = data.drop('Activity', axis = 1)
# Saving feature names for later use
data_list = list(data.columns)
# Convert to numpy array
data = np.array(data)

In [57]:
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(data, labels_activity, test_size = 0.25, random_state = 42)

In [58]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (9840, 31)
Training Labels Shape: (9840,)
Testing Features Shape: (3280, 31)
Testing Labels Shape: (3280,)


In [59]:
# Create a random forest classifier
clf = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)
clf.fit(train_features, train_labels)

RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=0)

In [60]:
# Print the name and gini importance of each feature
feature_list = []
for feature in zip(data_list, clf.feature_importances_):
    feature_list.append(feature)

In [61]:
#feature to dictionary
result = dict(feature_list)
#sorting features
result = dict(sorted(result.items(), key=lambda item: item[1], reverse=True))

In [62]:
result

{'timesincecasestart': 0.19192933906502527,
 'timesincelastevent': 0.17680087934245511,
 'Leucocytes': 0.10832547676019641,
 'timesincemidnight': 0.10748762051544142,
 'CRP': 0.10248153625918417,
 'LacticAcid': 0.07684186107144948,
 'open_cases': 0.05926251649823782,
 'Diagnose': 0.04259678382574944,
 'Age': 0.035795212925727744,
 'DiagnosticArtAstrup': 0.010051357695577625,
 'SIRSCritTachypnea': 0.009676325409415296,
 'DiagnosticUrinaryCulture': 0.009309281581782684,
 'DiagnosticUrinarySediment': 0.008903029119405914,
 'SIRSCritTemperature': 0.005972495348584318,
 'SIRSCritHeartRate': 0.0057209253119712465,
 'DiagnosticECG': 0.005570025618974757,
 'SIRSCritLeucos': 0.004496041547956162,
 'DiagnosticXthorax': 0.004480640384928373,
 'Hypotensie': 0.004443321161671445,
 'DisfuncOrg': 0.003992081370606646,
 'Infusion': 0.003873774668361125,
 'DiagnosticLacticAcid': 0.0038473709870625794,
 'DiagnosticBlood': 0.003304403833381248,
 'Oligurie': 0.0029348275398772645,
 'DiagnosticSputum': 0.0

In [63]:
# Create a selector object that will use the random forest classifier to identify
# features that have an importance of more than 0.15
sfm = SelectFromModel(clf, threshold=0.01)

# Train the selector
sfm.fit(train_features, train_labels)

SelectFromModel(estimator=RandomForestClassifier(n_estimators=1000, n_jobs=-1,
                                                 random_state=0),
                threshold=0.01)

In [75]:
# Print the names of the most important features
data_list_important = []
for feature_list_index in sfm.get_support(indices=True):
    print(data_list[feature_list_index])
    data_list_important.append(data_list[feature_list_index])

Diagnose
DiagnosticArtAstrup
Age
CRP
LacticAcid
Leucocytes
timesincemidnight
timesincelastevent
timesincecasestart
open_cases


In [65]:
# Transform the data to create a new dataset containing only the most important features
# Note: We have to apply the transform to both the training X and test X data.
X_important_train = sfm.transform(train_features)
X_important_test = sfm.transform(test_features)

In [66]:
# Create a new random forest classifier for the most important features
clf_important = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)

# Train the new classifier on the new dataset containing the most important features
clf_important.fit(X_important_train, train_labels)

RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=0)

In [69]:
# Apply The Full Featured Classifier To The Test Data
y_pred = clf.predict(test_features)

# View The Accuracy Of Our Full Feature (4 Features) Model
accuracy_score(test_labels, y_pred)

0.510670731707317

In [70]:
# Apply The Full Featured Classifier To The Test Data
y_important_pred = clf_important.predict(X_important_test)

# View The Accuracy Of Our Limited Feature (2 Features) Model
accuracy_score(test_labels, y_important_pred)

0.561280487804878

In [76]:
data_list_important

['Diagnose',
 'DiagnosticArtAstrup',
 'Age',
 'CRP',
 'LacticAcid',
 'Leucocytes',
 'timesincemidnight',
 'timesincelastevent',
 'timesincecasestart',
 'open_cases']